In [ ]:
!pip install -U efficientnet
!pip install -U pycm
from pycm import ConfusionMatrix

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams["figure.figsize"] = (10,10)
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler

from keras.preprocessing.image import ImageDataGenerator

from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle
import random
from sklearn.metrics import confusion_matrix




from efficientnet.tfkeras import EfficientNetB0
from efficientnet.tfkeras import EfficientNetB1
from efficientnet.tfkeras import EfficientNetB2
from efficientnet.tfkeras import EfficientNetB3
from efficientnet.tfkeras import EfficientNetB4
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121
import itertools
from mlxtend.plotting import plot_confusion_matrix

%pylab inline

In [ ]:
data_set_path="../input/augisic2019/augmented_isic_2019/images/"
ground_truth="../input/augisic2019/augmented_isic_2019/train.csv"


In [ ]:
train_data=pd.read_csv(ground_truth)
train_data.drop('Unnamed: 0',axis=1,inplace=True)
train_data.head()

In [ ]:
train_data.head()

In [ ]:
train_data=train_data.fillna(0)

In [ ]:
cols=train_data.columns
cols

In [ ]:
print("Total images : "+ str(len(train_data)))
cols=['NV','BCC','AK','BKL','DF','VASC','SCC','MEL']
print("Total class : "+ str(len(cols))  )    

y=np.array([])
for i in range(0,len(cols)):
    y=np.append(y,len(train_data[train_data[cols[i]]>0]))


plt.bar(cols,y)
plt.show()            

In [ ]:
data=train_data[train_data[cols[0]]>0].iloc[:1500]
for i in range(1,len(cols)):
    data=data.append(train_data[train_data[cols[i]]>0].iloc[:1500])


In [ ]:
# data=train_data[train_data[cols[0]]>0]
# for i in range(1,len(cols)):
#     if i==3:
#         data=data.append(train_data[train_data[cols[i]]>0].iloc[:int(len(train_data[train_data[cols[5]]>0]))])
#     elif i==2:    
#         data=data.append(train_data[train_data[cols[i]]>0].iloc[:int(len(train_data[train_data[cols[5]]>0]))])
#     elif i==7:    
#         data=data.append(train_data[train_data[cols[i]]>0].iloc[:int(len(train_data[train_data[cols[5]]>0]))])
#     else:
#         data=data.append(train_data[train_data[cols[i]]>0])

In [ ]:
train_data=data

In [ ]:
train_data.head()

In [ ]:
print("Total images : "+ str(len(train_data)))
cols=['NV','BCC','AK','BKL','DF','VASC','SCC','MEL']
print("Total class : "+ str(len(cols))  )    

y=np.array([])
for i in range(0,len(cols)):
    y=np.append(y,len(train_data[train_data[cols[i]]>0]))


plt.bar(cols,y)
plt.show()            

In [ ]:
train_data=train_data.sample(frac=1)

In [ ]:
train_data.head()

In [ ]:
train=train_data[train_data[cols[0]]>0].iloc[0:int(len(train_data[train_data[cols[0]]>0])*0.85)]
test_img=train_data[train_data[cols[0]]>0].iloc[int(len(train_data[train_data[cols[0]]>0])*0.85):int(len(train_data[train_data[cols[0]]>0])*0.95)]
val=train_data[train_data[cols[0]]>0].iloc[int(len(train_data[train_data[cols[0]]>0])*0.95):]

for i in range(1,len(cols)):
    train=train.append(train_data[train_data[cols[i]]>0].iloc[0:int(len(train_data[train_data[cols[i]]>0])*0.85)])
    test_img=test_img.append(train_data[train_data[cols[i]]>0].iloc[int(len(train_data[train_data[cols[i]]>0])*0.85):int(len(train_data[train_data[cols[i]]>0])*0.95)])
    val=val.append(train_data[train_data[cols[i]]>0].iloc[int(len(train_data[train_data[cols[i]]>0])*0.95):])

In [ ]:
print("Images in test : "+str(len(test_img)))
print("Images in train : "+str(len(train)))
print("Images in val : "+str(len(val)))

total_val=np.array([])
total_train=np.array([])
total_test=np.array([])
for i in range(0,len(cols)):
      total_train=np.append(total_train,int(len(train[train[cols[i]]>0])))
      total_val=np.append(total_val,int(len(val[val[cols[i]]>0])))
      total_test=np.append(total_test,int(len(test_img[test_img[cols[i]]>0])))
        
fig, a = plt.subplots(1, 3,figsize=(15,7))
fig.suptitle('Test train validation split')
a[0].bar(cols,total_train)
a[1].bar(cols,total_val)
a[2].bar(cols,total_test)
plt.show() 



In [ ]:
train.reset_index(drop=True, inplace=True)
test_img.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [ ]:
train['image']=data_set_path + train['image'] +".jpg"
val['image']=data_set_path + val['image'] +".jpg"
test_img['image']=data_set_path+ test_img['image'] +".jpg"

In [ ]:
train['image'].iloc[0]

In [ ]:
test_img.head()

In [ ]:
test_i=test_img.iloc[:2]

test_i['image'].iloc[0]

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams["figure.figsize"] = (10,10)
img = mpimg.imread('../input/augisic2019/augmented_isic_2019/images/ISIC_0000000.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
train=train.sample(frac=1)

In [ ]:

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
    width_shift_range=0.001,
    height_shift_range=0.001,
    brightness_range=None,
    shear_range=0.001,
    zoom_range=0.001,
    channel_shift_range=0.001,
    fill_mode="nearest")

train_generator = train_datagen.flow_from_dataframe(train,
                                                    x_col='image',
                                                    y_col=cols,
                                                    target_size = (380,380), 
                                                    class_mode = 'raw',
                                                    batch_size = 8
                                                    
                                                   )

val_generator = train_datagen.flow_from_dataframe(val,
                                                  x_col='image',
                                                  y_col=cols,
                                                  target_size = (380,380),
                                                  class_mode = 'raw',
                                                  batch_size = 4
                                                 
                                                  )
#train_datagen.fit(val_generator)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(test_img,
                                                  x_col='image',
                                                  y_col=cols,
                                                  target_size = (380,380),
                                                  class_mode = 'raw',
                                                  batch_size = 1,
                                              
                                                 )

In [ ]:
test_generator.labels[0]

In [ ]:



# EP=90
# LR=0.0001
# DR=LR/EP
# MM=0.8
# K=0.6

# def build_lrfn(lr_start=0.00001, lr_max=0.000075, lr_min=0.000001, lr_rampup_epochs=20, lr_sustain_epochs=0, lr_exp_decay=.8):
#     lr_max = lr_max * 8
#     def lrfn(epoch):
#         if epoch < lr_rampup_epochs:
#             lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
#         elif epoch < lr_rampup_epochs + lr_sustain_epochs:
#             lr = lr_max
#         else:
#             lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
#         return lr
#     return lrfn

# lrfn = build_lrfn()

# LRD=tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

# callbacks_list=[LRD]



# "-- KEEP THIS IN MIND--"
# #This line below give error because a non sense reason 
# #OPT=keras.optimizers.Adam(lr=lr_schedule,decy_rate=DR)

# "--CORRECT WAY WITH LEARNING RATE SCHEDULE--"
# OPT=keras.optimizers.Adam(learning_rate=LR)




def build_lrfn():

    def lrfn(epoch):
        if epoch <1 :
          lr=0.00009
        elif epoch < 7:
          lr=0.00008 
        elif epoch < 10:
          lr=0.00003
        elif epoch <= 20:
          lr=0.000009
        elif epoch > 20:
          lr=0.0000009 

       
        return lr
    return lrfn
lrfn = build_lrfn()
LRD=tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)




from keras.utils.vis_utils import plot_model



model = keras.Sequential()
base_model=EfficientNetB2(input_shape=(380,380, 3), drop_connect_rate=0.5,include_top=False)

model.add(base_model)

print(base_model.summary())
plot_model(base_model, to_file='base_model_plot.png', show_shapes=True, show_layer_names=True)
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(8, activation='sigmoid')) 

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

early_stopping_monitor = EarlyStopping(patience=7,monitor='val_accuracy')
model_checkpoint_callback = ModelCheckpoint(filepath='best_model.hdf5',
                                            save_weights_only=True,
                                            monitor='val_accuracy',
                                            mode='auto',
                                            save_best_only=True,
                                            verbose=1)

print(model.summary())


In [ ]:
model.input_shape

In [ ]:
model.losses

In [ ]:

print(model.summary())

In [ ]:
OPT=keras.optimizers.Adam(learning_rate=0.000009)
model.compile(loss='categorical_crossentropy', optimizer=OPT, metrics=['accuracy', 'AUC'])
history=model.fit(train_generator, epochs=25, steps_per_epoch=train_generator.n//train_generator.batch_size,validation_data=val_generator, validation_steps=val_generator.n//val_generator.batch_size,callbacks=[early_stopping_monitor,model_checkpoint_callback])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('Training and validation accuracy.png')
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.savefig('Training and validation loss.png')

In [ ]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
model.load_weights("best_model.hdf5")

In [ ]:
y=np.concatenate([test_generator.next()[1] for i in range(test_generator.__len__())])
prediction= model.predict(test_generator, verbose=2)

true_labels=y

from sklearn.metrics import roc_curve,auc
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(cols)):
    fpr[i], tpr[i], _ = roc_curve(true_labels[:,i], prediction[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.rcParams["figure.figsize"] = (15,15)
plt.figure()
# Plot of a ROC curve for a specific class
for i in range(len(cols)):
    plt.plot(fpr[i], tpr[i], label='ROC curve  (area = %0.2f)' + cols[i]% roc_auc[i])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
plt.savefig('ROC curve.png')


In [ ]:
for i in range(len(cols)):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve  (area = %0.2f)' + cols[i]% roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    plt.savefig('ROC curve ' +cols[i]+'.png')


In [ ]:
true_labels=np.argmax(y, axis=-1)
prediction=np.argmax(prediction, axis=-1)
cm = confusion_matrix(y_true=true_labels, y_pred=prediction)

cm_plot_labels = cols
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
print(metrics.classification_report(true_labels,prediction))

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(cm,
                                colorbar=True,
                                show_absolute=False,
                                show_normed=True,
                               figsize=(15,10))

plt.show()
plt.savefig('normal cm.png')

from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(cm,
                                colorbar=True,
                                show_absolute=True,
                                show_normed=False,
                               figsize=(15,10))

plt.show()
plt.savefig('absolute cm.png')

In [ ]:
print(ConfusionMatrix(actual_vector=true_labels,predict_vector=prediction))